In [ ]:
import os
import torch
import pickle
import numpy as np
import cv2
import pandas as pd
from datetime import datetime
from PIL import Image
from facenet_pytorch import InceptionResnetV1, MTCNN
from torchvision import transforms

# Load MTCNN for face detection
mtcnn = MTCNN()

# Load the trained model
model_path = "face_classifier.pkl"
with open(model_path, "rb") as f:
    data = pickle.load(f)
    classifier = data["model"]
    label_map = data["label_map"]  # Mapping of label indices to names

# Load FaceNet model
facenet = InceptionResnetV1(pretrained='vggface2').eval()

def load_image(img):
    transform = transforms.Compose([
        transforms.Resize((160, 160)),
        transforms.ToTensor()
    ])
    return transform(img).unsqueeze(0)

def recognize_face(frame):
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    boxes, _ = mtcnn.detect(img_rgb)

    if boxes is not None:
        for box in boxes:
            x1, y1, x2, y2 = map(int, box)
            
            # Ensure valid face region
            if x1 < 0 or y1 < 0 or x2 > frame.shape[1] or y2 > frame.shape[0]:
                continue  # Skip invalid boxes

            face = frame[y1:y2, x1:x2]

            # Ensure face is not empty
            if face.size == 0:
                continue  

            face = Image.fromarray(cv2.cvtColor(face, cv2.COLOR_BGR2RGB))
            face_tensor = load_image(face)

            with torch.no_grad():
                embedding = facenet(face_tensor).numpy().flatten()
                recognized_name = predict_identity(embedding)
                mark_attendance(recognized_name)

                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, recognized_name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    return frame


def predict_identity(embedding):
    predicted_label = classifier.predict([embedding])[0]
    return label_map.get(predicted_label, "Unknown")  # Convert label to name


def mark_attendance(name):
    file_path = "attendance.csv"
    now = datetime.now()
    dt_string = now.strftime('%Y-%m-%d %H:%M:%S')
    
    if not os.path.exists(file_path) or os.stat(file_path).st_size == 0:
        df = pd.DataFrame(columns=["Name", "Time"])
    else:
        try:
            df = pd.read_csv(file_path)
        except pd.errors.EmptyDataError:
            df = pd.DataFrame(columns=["Name", "Time"])
    
    new_entry = pd.DataFrame({"Name": [name], "Time": [dt_string]})
    df = pd.concat([df, new_entry], ignore_index=True)
    df.to_csv(file_path, index=False)

def main():
   # cap = cv2.VideoCapture("http://192.168.0.108:4747/video")
    cap = cv2.VideoCapture(0)  # Old webcam code
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = recognize_face(frame)
        cv2.imshow("Face Recognition Attendance", frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()